# Training the Neural Network

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets, models, transforms
import os

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Enable GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True

# Data augmentation and normalization for training
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load dataset
data_dir = 'data/'
image_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])

# Perform 5-fold cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(skf.split(np.arange(len(image_dataset)), image_dataset.targets)):
    print(f'Fold {fold + 1}/{num_folds}')
    print('-' * 10)

    # Create DataLoader for training set
    train_sampler = SubsetRandomSampler(train_index)
    train_loader = DataLoader(image_dataset, batch_size=32, sampler=train_sampler, num_workers=4)

    # Create DataLoader for validation set
    val_sampler = SubsetRandomSampler(val_index)
    val_loader = DataLoader(image_dataset, batch_size=32, sampler=val_sampler, num_workers=4)

    # Model definition
    model_ft = models.resnet18(pretrained=True)
    for param in model_ft.parameters():
        param.requires_grad = False

    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, len(image_dataset.classes))
    model_ft = model_ft.to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

    # Initialize variables to track the best model
    best_acc = 0.0
    best_model = None

    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model_ft.train()
                dataloader = train_loader
            else:
                model_ft.eval()
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer_ft.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model_ft(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer_ft.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                exp_lr_scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Check if validation accuracy is the best, if yes, save the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = model_ft

    # Save the best model for the current fold
    if best_model is not None:
        model_save_path = f'best_model_fold{fold + 1}.pth'
        torch.save(best_model.state_dict(), model_save_path)
        print(f"Best model for fold {fold + 1} saved at: {model_save_path}")

print('Training complete')


Fold 1/5
----------
Epoch 0/9
----------
train Loss: 1.7127 Acc: 0.2326
val Loss: 0.3742 Acc: 0.0842
Epoch 1/9
----------
train Loss: 1.3665 Acc: 0.3817
val Loss: 0.3218 Acc: 0.1040
Epoch 2/9
----------
train Loss: 1.2068 Acc: 0.4341
val Loss: 0.2854 Acc: 0.1147
Epoch 3/9
----------
train Loss: 1.1184 Acc: 0.4621
val Loss: 0.2681 Acc: 0.1223
Epoch 4/9
----------
train Loss: 1.0559 Acc: 0.4762
val Loss: 0.2570 Acc: 0.1185
Epoch 5/9
----------
train Loss: 1.0147 Acc: 0.4922
val Loss: 0.2499 Acc: 0.1232
Epoch 6/9
----------
train Loss: 0.9697 Acc: 0.4977
val Loss: 0.2417 Acc: 0.1274
Epoch 7/9
----------
train Loss: 0.9275 Acc: 0.5143
val Loss: 0.2335 Acc: 0.1333
Epoch 8/9
----------
train Loss: 0.9307 Acc: 0.5236
val Loss: 0.2325 Acc: 0.1307
Epoch 9/9
----------
train Loss: 0.9318 Acc: 0.5160
val Loss: 0.2341 Acc: 0.1303
Best model for fold 1 saved at: best_model_fold1.pth
Fold 2/5
----------
Epoch 0/9
----------
train Loss: 1.7208 Acc: 0.2347
val Loss: 0.3759 Acc: 0.0878
Epoch 1/9
------

# Testing the Neural Network

In [32]:
# Define the path to the test data
test_data_dir = 'data'

# Load the test dataset
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=data_transforms['test'])

# Create a DataLoader for the test set
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load the best model
best_model_path = 'best_model_fold4.pth'
model_ft = models.resnet18(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 12)
model_ft.load_state_dict(torch.load(best_model_path, map_location='cpu'))
model_ft.eval()

# Move the model to the device (CPU in this case)
device = torch.device("cpu")
model_ft = model_ft.to(device)

# Make predictions on the test set
all_predictions = []
all_image_paths = []

with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        all_predictions.extend(preds.cpu().numpy())

# Get class names from the training dataset
class_names = image_dataset.classes

# Create a DataFrame with image paths and corresponding class predictions
results_df = pd.DataFrame({
    'Image_Path': [path for path, _ in test_dataset.imgs],
    'Predicted_Class': [class_names[pred] for pred in all_predictions]
})

# Save the results to a CSV file
results_csv_path = 'Submission Results.csv'
results_df.to_csv(results_csv_path, index=False)

print(f"Results saved to {results_csv_path}")

Results saved to Submission Results.csv
